In [1]:
import QpixAsicArray as qparray
from QpixAsic import PixelHit, ProcQueue
import random
import numpy as np
random.seed(2)

In [4]:
qpa = qparray.QpixAsicArray(3, 3, debug=0.0, deltaT=2e-6)
qpa._daqNode._debugLevel = 0
qpa.Calibrate(0.2)
# qpa.Calibrate(0.2)
print(qpa._daqNode.daqHits)

performing calibration..
received data! 265
received data! 549
received data! 547
received data! 789
received data! 766
received data! 1080
received data! 789
received data! 1107
received data! 1293
calibration complete in 100043 steps!
current time is 0.2000000000001587
9


In [10]:
qpa.Calibrate(0.2)

performing calibration..
received data! 49021734
received data! 52756085
received data! 52376488
received data! 50481714
received data! 48081970
received data! 52194601
received data! 50333113
received data! 53747817
received data! 49971511
calibration complete in 100043 steps!
current time is 1.200005999992832


In [11]:
guessFrq = {}
for k, v in qpa._daqNode.pixelData.items():
    asic = k
    # print(asic)
    daqTicks = []
    asicTicks = []
    for daqTick, asicTick in v:
        daqTicks.append(daqTick)
        asicTicks.append(asicTick)
    print("asic ticks:", asicTicks)
    guessFrq[asic] = (asicTicks[1] - asicTicks[0]) / (daqTicks[1] - daqTicks[0])  * qpa._daqNode.fOsc
for asic in qpa:
    key = f"({asic.row},{asic.col})"
    print(f"{key} percent diff: {(guessFrq[key] - asic.fOsc) / asic.fOsc * 100:1.2E}")

asic ticks: [265, 24510999, 49021734]
asic ticks: [549, 26378317, 52756085]
asic ticks: [547, 26188517, 52376488]
asic ticks: [789, 25241252, 50481714]
asic ticks: [766, 24041368, 48081970]
asic ticks: [1080, 26097841, 52194601]
asic ticks: [789, 25166951, 50333113]
asic ticks: [1107, 26874462, 53747817]
asic ticks: [1293, 24986402, 49971511]
(0,0) percent diff: -1.44E-06
(0,1) percent diff: 3.07E-07
(0,2) percent diff: 1.50E-06
(1,0) percent diff: -2.13E-06
(1,1) percent diff: -5.53E-07
(1,2) percent diff: 1.31E-06
(2,0) percent diff: 1.32E-06
(2,1) percent diff: 2.54E-07
(2,2) percent diff: -1.48E-06
